In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_M.tuberculosis_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(2097, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)
    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_M.tuberculosis_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_M.tuberculosis_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


118/118 ━━━━━━━━━━━━━━━━━━━━ 23s 70ms/step - AUC: 0.6455 - categorical_accuracy: 0.6483 - distill_loss1: 0.0256 - distill_loss2: 0.0766 - loss: 0.5000 - student_loss: 0.6446 - val_AUC: 0.5842 - val_categorical_accuracy: 0.5810 - val_loss: 0.5000 - val_student_loss: 0.6399
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - AUC: 0.7090 - categorical_accuracy: 0.6608 - distill_loss1: 0.0229 - distill_loss2: 0.0698 - loss: 0.5000 - student_loss: 0.6083 - val_AUC: 0.6132 - val_categorical_accuracy: 0.5810 - val_loss: 0.5000 - val_student_loss: 0.6141
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - AUC: 0.7293 - categorical_accuracy: 0.6661 - distill_loss1: 0.0222 - distill_loss2: 0.0675 - loss: 0.5000 - student_loss: 0.5996 - val_AUC: 0.6309 - val_categorical_accuracy: 0.5857 - val_loss: 0.5000 - val_student_loss: 0.5943
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - AUC: 0.7769 - categorical_accuracy: 0.6994 - distill_loss1: 0.0215 - distill_loss2: 0.0659 - loss: 0.5000 

...

118/118 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - AUC: 0.5184 - categorical_accuracy: 0.5154 - distill_loss1: 0.0307 - distill_loss2: 0.0829 - loss: 0.5000 - student_loss: 0.6748 - val_AUC: 0.7575 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.6075
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - AUC: 0.7091 - categorical_accuracy: 0.6753 - distill_loss1: 0.0235 - distill_loss2: 0.0716 - loss: 0.5000 - student_loss: 0.6237 - val_AUC: 0.7704 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.5784
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - AUC: 0.7248 - categorical_accuracy: 0.6684 - distill_loss1: 0.0226 - distill_loss2: 0.0691 - loss: 0.5000 - student_loss: 0.6100 - val_AUC: 0.7791 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.5715
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - AUC: 0.7618 - categorical_accuracy: 0.6879 - distill_loss1: 0.0222 - distill_loss2: 0.0668 - loss: 0.5000

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 75ms/step - AUC: 0.6209 - categorical_accuracy: 0.6097 - distill_loss1: 0.0268 - distill_loss2: 0.0783 - loss: 0.5000 - student_loss: 0.6562 - val_AUC: 0.7320 - val_categorical_accuracy: 0.7225 - val_loss: 0.5000 - val_student_loss: 0.6322
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - AUC: 0.7079 - categorical_accuracy: 0.6728 - distill_loss1: 0.0232 - distill_loss2: 0.0722 - loss: 0.5000 - student_loss: 0.6268 - val_AUC: 0.7595 - val_categorical_accuracy: 0.7225 - val_loss: 0.5000 - val_student_loss: 0.6162
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - AUC: 0.7454 - categorical_accuracy: 0.6843 - distill_loss1: 0.0223 - distill_loss2: 0.0692 - loss: 0.5000 - student_loss: 0.6109 - val_AUC: 0.7676 - val_categorical_accuracy: 0.7225 - val_loss: 0.5000 - val_student_loss: 0.6256
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - AUC: 0.7491 - categorical_accuracy: 0.6793 - distill_loss1: 0.0210 - distill_loss2: 0.0662 - loss: 0.5000 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW

k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Khởi tạo lại distiller cho từng fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Re-initialize the distiller for each fold
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Train model
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)

    # Save result of each fold
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_M.tuberculosis_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_M.tuberculosis_KD2_Succ1.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - AUC: 0.6053 - categorical_accuracy: 0.5846 - loss: 0.8313
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - AUC: 0.6074 - categorical_accuracy: 0.5926 - loss: 0.8127
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - AUC: 0.6622 - categorical_accuracy: 0.6276 - loss: 0.7383
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - AUC: 0.6350 - categorical_accuracy: 0.5983 - loss: 0.7493
Epoch 5/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - AUC: 0.6818 - categorical_accuracy: 0.6329 - loss: 0.6747
Epoch 6/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - AUC: 0.6764 - categorical_accuracy: 0.6435 - loss: 0.6727
Epoch 7/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - AUC: 0.6616 - categorical_accuracy: 0.6318 - loss: 0.6847
Epoch 8/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - AUC: 0.6787 - categorical_accuracy: 0.6475 - loss: 0.6670
Epoch 9/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - AUC: 0.6670 - categorical_accuracy: 0.6228 - loss: 0

...

118/118 ━━━━━━━━━━━━━━━━━━━━ 18s 53ms/step - AUC: 0.6004 - categorical_accuracy: 0.5904 - distill_loss1: 0.0098 - distill_loss2: 0.1017 - loss: 0.5000 - student_loss: 0.6660 - val_AUC: 0.7099 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.6257
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - AUC: 0.6955 - categorical_accuracy: 0.6666 - distill_loss1: 0.0060 - distill_loss2: 0.0929 - loss: 0.5000 - student_loss: 0.6308 - val_AUC: 0.7399 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.5972
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - AUC: 0.7153 - categorical_accuracy: 0.6601 - distill_loss1: 0.0052 - distill_loss2: 0.0894 - loss: 0.5000 - student_loss: 0.6144 - val_AUC: 0.7540 - val_categorical_accuracy: 0.6986 - val_loss: 0.5000 - val_student_loss: 0.6286
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - AUC: 0.7488 - categorical_accuracy: 0.6814 - distill_loss1: 0.0046 - distill_loss2: 0.0861 - loss: 0.5000

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - AUC: 0.5340 - categorical_accuracy: 0.5312 - loss: 0.8465
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - AUC: 0.6474 - categorical_accuracy: 0.6066 - loss: 0.7284
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - AUC: 0.5821 - categorical_accuracy: 0.5604 - loss: 0.7663
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - AUC: 0.6274 - categorical_accuracy: 0.5993 - loss: 0.7323
Epoch 5/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - AUC: 0.6492 - categorical_accuracy: 0.6043 - loss: 0.6975
Epoch 6/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - AUC: 0.6598 - categorical_accuracy: 0.6195 - loss: 0.6757
Epoch 7/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.6658 - categorical_accuracy: 0.6233 - loss: 0.6666
Epoch 8/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - AUC: 0.6566 - categorical_accuracy: 0.6140 - loss: 0.6734
Epoch 9/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - AUC: 0.6613 - categorical_accuracy: 0.6124 - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


118/118 ━━━━━━━━━━━━━━━━━━━━ 17s 53ms/step - AUC: 0.5835 - categorical_accuracy: 0.5740 - distill_loss1: 0.0084 - distill_loss2: 0.1017 - loss: 0.5000 - student_loss: 0.6668 - val_AUC: 0.7421 - val_categorical_accuracy: 0.7225 - val_loss: 0.5000 - val_student_loss: 0.6214
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - AUC: 0.7029 - categorical_accuracy: 0.6598 - distill_loss1: 0.0071 - distill_loss2: 0.0901 - loss: 0.5000 - student_loss: 0.6204 - val_AUC: 0.7606 - val_categorical_accuracy: 0.7225 - val_loss: 0.5000 - val_student_loss: 0.6109
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - AUC: 0.7265 - categorical_accuracy: 0.6591 - distill_loss1: 0.0067 - distill_loss2: 0.0877 - loss: 0.5000 - student_loss: 0.6094 - val_AUC: 0.7723 - val_categorical_accuracy: 0.7273 - val_loss: 0.5000 - val_student_loss: 0.5909
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - AUC: 0.7414 - categorical_accuracy: 0.6682 - distill_loss1: 0.0073 - distill_loss2: 0.0863 - loss: 0.5000